# Quantum Computing Hints for the coursework 



##  Installing pennylane on the University machines and your laptop

There are instructions on how to install pennylane on your computer at: https://pennylane.ai/install/

In the annaconda prompt or terminal on a Mac type 

pip install pennylane --upgrade

If you have spyder running you may need to restart spyder to access the library.

You will have to run this command everytime you work on a new PC in the lab.

There may be  additional complications with the University PCs,because of administrative rights. 




##  Question Quantum Phase Estimation

The aim of this section is to numerically investigate the quantum phase estimation algorithm. This is an algorithm to estimate the eigenvalue of a unitary matrix. We will first use a simple matrix with known eigenvalues.

* Background to Quantum Phase Estimation https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm

* See the Pennylane tutorial https://pennylane.ai/qml/demos/tutorial_qpe/

We want to estimate the phase $\phi$

$$
U \mid \psi \rangle = e^{2 i \pi \phi } \mid \psi \rangle
$$




In this question, we apply a 1 by 1 complex phase  $ e^{2 i \pi \phi }$ where $\phi $ is a known parameter.

To test the quantum phase estimation we input a known phase $\phi$
We can then check that the Quantum Phase Estimation can reproduce the known $\phi$.

The noise code should go in the function circuit_qpe


## input parameters
phi_input : The input phase $\phi$

measure_qubit : The number of qubits to estimate the phase


In [ ]:
phi_input = 0.2
measure_qubit = 5

In [ ]:
import pennylane as qml
import numpy as np

## This function applies the phase phi
def U(wires,phi):
    return qml.PhaseShift(2 * np.pi * phi, wires=wires)

In [ ]:
nwires = measure_qubit + 1
dev = qml.device('default.mixed', wires=nwires)

## Below is a code to compute the phase using quantum phase estimation
@qml.qnode(dev)
def circuit_qpe(estimation_wires,phi):
    # initialize to state |1>
    qml.PauliX(wires=0)

    for wire in estimation_wires:
        qml.Hadamard(wires=wire)

    qml.ControlledSequence(U(wires=0,phi=phi), control=estimation_wires)
    # The noise code should go here

    
    qml.adjoint(qml.QFT)(wires=estimation_wires)

    return qml.probs(wires=estimation_wires)

In [ ]:
estimation_wires = range(1, measure_qubit)

In [ ]:


# https://stackoverflow.com/questions/13662933/how-to-convert-a-string-representing-a-binary-fraction-to-a-number-in-python
def frac_bin_str_to_float(num):
    """Assuming num to be a string representing
    the fractional part of a binary number with
    no integer part, return num as a float."""
    result = 0
    ex = 2.0
    for c in num:
        if c == '1':
            result += 1/ex 
        ex *= 2
    return result

## Plot the circuit

In [ ]:
#  https://docs.pennylane.ai/en/stable/introduction/circuits.html

import matplotlib.pyplot as plt
qml.drawer.use_style("black_white")
fig, ax = qml.draw_mpl( circuit_qpe)(estimation_wires,0.2)
    
plt.show()

In [ ]:
import matplotlib.pyplot as plt

estimation_wires = range(1, measure_qubit)

results = circuit_qpe(estimation_wires,phi_input)

bit_strings = [f"0.{x:0{len(estimation_wires)}b}" for x in range(len(results))]

p_thresh = 0.4 
print("Values with probability > ", p_thresh) 
for b_, p_ in zip(bit_strings, results):
    if p_ > p_thresh  :
        phase_est_dec = frac_bin_str_to_float(b_[2:])
        print("binary=",b_, "decimal=", phase_est_dec ,)

plt.bar(bit_strings, results)
plt.xlabel("phase")
plt.ylabel("probability")
plt.xticks(rotation="vertical")
plt.subplots_adjust(bottom=0.3)
plt.title(r"Input phase  $\phi= $" + str(phi_input) )

plt.show()

The result for the phase is in the form of binary decimal notation  
$ x = 0. \phi_1 \phi_2 \phi_3 $   

where $\phi_j$ is 1 or  0

$x = \phi_1 / 2 + \phi_2 / 4 + \phi_3 / 8 $ 
